# Flux comparison
In FIG. 4 , the authors compared subsystems by selecting the most de-regulated reactions across cell types (LG and HG). I think They optimized to max biomass each of the 7 models  (they generated 7 FBA solutions) and then they computed the average of each flux across cell lines of the same family.

However, they selected one of the many solutions that respect max biomass growth. I suggest to:

1. Impose biomass at its upper bound for each model (LB = UB*0.90 to avoid solver numerical issues)
2. Run flux sampling with OPTGP (cobrapy) with thinning = 100 and 1k samples per cell line
3. DO not 'summarize' flux probability distributions with a simple average, but use more advanced methods. The objective here is to identify the most different reactions across the two cancer families. You could run non parametric statistical tests such as mann-whitney to check if two probability distributions are significantly different or not. You have 3 cells vs 2 cells (all pair combinations), so you could perform this test only on reactions belonging to core subsystems such as glycolysis, TCA cycle pentophosphate etc.. in order to redure the number of compared distribution per cell couple.
4. Once you identified the top-n most different probability distributions (reaction fluxes) across cells of different type, you could plot them with boxplots as the authors did 
5. It might be interesting to check if we have 'less differences' in distributions of cells belonging to the same family.

In [30]:

from cobra.io import load_model
from cobra.sampling import sample
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests

import pandas as pd
from cobra.io.json import load_json_model
import numpy as np
from cobra.flux_analysis import flux_variability_analysis
from cobra.flux_analysis import single_reaction_deletion
import math
from tqdm import tqdm

from cobra.io import read_sbml_model
from mewpy.simulation import get_simulator
from mewpy.simulation import set_default_solver
set_default_solver('gurobi')

from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests

# 1. Load Models

In [3]:
models = {
    "LG_59M": lg1_model, "LG_HEYA8": lg2_model, "LG_OV56": lg3_model,
    "HG_CAOV3": hg1_model, "HG_COV318": hg2_model, "HG_OAW28": hg2_model
}

NameError: name 'lg1_model' is not defined

In [29]:
model = load_json_model('./data/Recon3D.json')
model.solver = 'gurobi' #you can use cplex or glpk as you whish, but gurobi and cplex are usually faster

## 2. Map reactions to their subsystem

## Pathways
- Glycolysis
- TCA cycle
- PPP
- Amino acid metabolism
- Nucleotide metabolism
- Fatty acid metabolism
- Lipid metabolism

### 1. Impose biomass UB and LB=UP x 0.9

In [37]:
def constrain_to_near_opt(model):
    solution = model.optimize()
    v_opt = solution.fluxes["BIOMASS_reaction"]
    rxn = model.reactions.get_by_id("BIOMASS_reaction")
    rxn.lower_bound = 0.9 * v_opt   # 90% of optimal growth
    rxn.upper_bound = v_opt         # at most optimal growth


### 2. Run flux sampling with OPTGP (cobrapy) with thinning = 100 and 1k samples per cell line

In [41]:
all_samples = {}

for id,model in models.items():
    constrain_to_near_opt(model)
    S= sample(model, n=1000, method="optgp", thinning=100, seed=42)
    all_samples[id]=S 

all_samples

AttributeError: 'Model' object has no attribute 'items'

### 3. DO not 'summarize' flux probability distributions with a simple average, but use more advanced methods. The objective here is to identify the most different reactions across the two cancer families. 

In [16]:
subsystem_dict = {}

for r in model.reactions:
    # get subsystem safely
    s = (getattr(r, "subsystem", "") or "").strip()

    # if it contains "/", keep only the first part
    if "/" in s:
        s = s.split("/")[0].strip()   # take text before "/" and remove spaces

    s = s.lower()

    # skip empty subsystems
    if not s:
        continue

    # add reaction to that subsystem list
    subsystem_dict.setdefault(s, []).append(r.id)

In [20]:
category_keywords = {
    "Glycolysis": [
        "glycolysis", "gluconeogenesis"
    ],
    
    "TCA": [
        "tca", "citric acid", "krebs"
    ],
    
    "Pentose Phosphate Pathway": [
        "pentose phosphate", "ppp"
    ],
    
    "Amino acid metabolism": [
        "amino acid",
        "tyrosine", "phenylalanine", "tryptophan",
        "lysine", "leucine", "isoleucine", "valine",
        "methionine", "cysteine", "serine", "threonine",
        "histidine", "arginine", "ornithine", "proline",
        "glutamate", "glutamine", "aspartate", "asparagine",
        "alanine", "glycine"
    ],
    
    "Nucleotide metabolism": [
        "nucleotide", "purine", "pyrimidine", "deoxynucleotide", "dntp"
    ],
    
    "Fatty acid metabolism": [
        "fatty acid", "beta oxidation", "beta-oxidation",
        "acyl", "acyl-coa"
    ],
    
    "Lipid metabolism": [
        "lipid", "phospholipid", "sphingolipid", "glycerolipid"
    ],
}

In [23]:
keys = list(subsystem_dict.keys())
print(keys)

['transport, mitochondrial', 'transport, extracellular', 'transport, lysosomal', 'extracellular exchange', 'vitamin d metabolism', 'transport, endoplasmic reticular', 'beta-alanine metabolism', 'glycine, serine, alanine, and threonine metabolism', 'methionine and cysteine metabolism', 'lysine metabolism', 'tryptophan metabolism', 'tyrosine metabolism', 'ubiquinone synthesis', 'taurine and hypotaurine metabolism', 'cytochrome metabolism', 'steroid metabolism', 'sphingolipid metabolism', 'o-glycan metabolism', 'blood group synthesis', 'glutamate metabolism', 'valine, leucine, and isoleucine metabolism', 'fatty acid oxidation', 'transport, peroxisomal', 'propanoate metabolism', 'transport, golgi apparatus', 'aminosugar metabolism', 'transport, nuclear', 'urea cycle', 'citric acid cycle', 'vitamin b2 metabolism', 'nucleotide interconversion', 'arginine and proline metabolism', 'purine synthesis', 'keratan sulfate synthesis', 'alanine and aspartate metabolism', 'n-glycan degradation', 'bile

In [24]:
selected = {cat: [] for cat in category_keywords}

In [25]:
for subsystem_name, rxns in subsystem_dict.items():
    name_lower = subsystem_name.lower()

    for category, keywords in category_keywords.items():
        if any(kw in name_lower for kw in keywords):
            selected[category].extend(rxns)

In [26]:
for cat, rxns in selected.items():
    print(f"{cat}: {len(set(rxns))} reactions")

Glycolysis: 42 reactions
TCA: 20 reactions
Pentose Phosphate Pathway: 41 reactions
Amino acid metabolism: 439 reactions
Nucleotide metabolism: 273 reactions
Fatty acid metabolism: 1200 reactions
Lipid metabolism: 369 reactions


In [27]:
for rxn_id in selected["Lipid metabolism"]:
    rxn = model.reactions.get_by_id(rxn_id)
    print(rxn_id, " → ", rxn.subsystem)

A4GALTc  →  Sphingolipid metabolism
A4GALTg  →  Sphingolipid metabolism
B3GALT3g  →  Sphingolipid metabolism
B3GALT42g  →  Sphingolipid metabolism
B3GNT31g  →  Sphingolipid metabolism
B3GNT34g  →  Sphingolipid metabolism
B3GNT37g  →  Sphingolipid metabolism
B3GNT39g  →  Sphingolipid metabolism
DHCRD2  →  Sphingolipid metabolism
DSAT  →  Sphingolipid metabolism
GALGT2  →  Sphingolipid metabolism
GALGT3  →  Sphingolipid metabolism
GAO1g  →  Sphingolipid metabolism
GBA  →  Sphingolipid metabolism
GBGT1  →  Sphingolipid metabolism
GLAl  →  Sphingolipid metabolism
GLB1  →  Sphingolipid metabolism
NAGAlby  →  Sphingolipid metabolism
SBPP3er  →  Sphingolipid metabolism
SGPL12r  →  Sphingolipid metabolism
SLCBK1  →  Sphingolipid metabolism
SMPD3g  →  Sphingolipid metabolism
SMS  →  Sphingolipid metabolism
SPHK21c  →  Sphingolipid metabolism
ST3GAL21g  →  Sphingolipid metabolism
ST3GAL22g  →  Sphingolipid metabolism
ST3GAL23g  →  Sphingolipid metabolism
ST6GALNAC21  →  Sphingolipid metabolism
S

In [34]:
fva_result_gly = flux_variability_analysis(
    model,
    fraction_of_optimum=0.9  ,  # 90% of optimal biomass,
    reaction_list=selected["Glycolysis"]
)

Set parameter Username
Set parameter LicenseID to value 2732828
Academic license - for non-commercial use only - expires 2026-11-04
Read LP format model from file /var/folders/0f/7pcrwybx3_zfsnzs2qjgxp8h0000gn/T/tmpqwm49o43.lp
Reading time = 0.02 seconds
: 5836 rows, 21201 columns, 80853 nonzeros
Set parameter Username
Set parameter LicenseID to value 2732828
Academic license - for non-commercial use only - expires 2026-11-04
Read LP format model from file /var/folders/0f/7pcrwybx3_zfsnzs2qjgxp8h0000gn/T/tmplqmwriou.lp
Reading time = 0.03 seconds
: 5836 rows, 21201 columns, 80853 nonzeros
Set parameter Username
Set parameter LicenseID to value 2732828
Academic license - for non-commercial use only - expires 2026-11-04
Read LP format model from file /var/folders/0f/7pcrwybx3_zfsnzs2qjgxp8h0000gn/T/tmpex877pey.lp
Reading time = 0.03 seconds
: 5836 rows, 21201 columns, 80853 nonzeros
Set parameter Username
Set parameter LicenseID to value 2732828
Academic license - for non-commercial use 

In [36]:
fva_result_gly

,minimum,maximum
ALCD2y,0.000000,1000.0
ALDD2xm,0.000000,1000.0
ALDD2y,0.000000,1000.0
CBPPer,0.000000,1000.0
DPGase,0.000000,1000.0
G6PPer,0.000000,1000.0
PDHm,0.000000,1000.0
PYK2,0.000000,1000.0
r0202,-1000.000000,1000.0
r0354,0.000000,1000.0


### 4. Use non-parametric test: You could run non parametric statistical tests such as mann-whitney to check if two probability distributions are significantly different or not. You have 3 cells vs 2 cells (all pair combinations), so you could perform this test only on reactions belonging to core subsystems such as glycolysis, TCA cycle pentophosphate etc.. in order to redure the number of compared distribution per cell couple.

In [ ]:
reactions_by_category = {
    "glycolysis": selected["Glycolysis"],
    "tca": selected["TCA"],
    "ppp": selected["Pentose Phosphate Pathway"],
    "amino_acid": selected["Amino acid metabolism"],
    "nucleotide": selected["Nucleotide metabolism"],
    "fatty_acid": selected["Fatty acid metabolism"],
    "lipid": selected["Lipid metabolism"],
}

- index = rxn_id
- columns = cell_id

In [42]:
def subsystem_df(models, subsystem, N=4):
    # 1. 

SyntaxError: incomplete input (869462133.py, line 2)